In [5]:
import pytse_client as tse
import numpy as np
import torch
import torch.nn as nn
# tickers = tse.download(symbols="all", write_to_csv=True)

Device = ("cuda"
          if torch.cuda.is_available()
          else "cpu"
)
print(f"Using {Device} device")


Using cuda device


In [6]:
sym="آپ"
ticker=tse.Ticker(sym)
length=ticker.history.shape[0]
train_length = int(np.floor(length*0.90))
test_length = length - train_length
days_history=100
days_predict=30

# Now we extract some portion of the first sequences of data for train
X_train=np.zeros([train_length - days_history - days_predict , days_history,5])
Y_train=np.zeros([train_length - days_history - days_predict , days_predict])
for i in range(train_length - days_history - days_predict):
    m = ticker.history.loc[i : i + days_history - 1].to_numpy()
    X_train[i,:,:] = np.array([(m[:,9]-m[:,8])/m[:,8] * 100 , m[:,7]/1e3 , m[:,6]/1e6 , m[:,2] , m[:,3]]).T
    m = ticker.history.loc[i + days_history : i + days_history - 1 + days_predict].to_numpy()
    Y_train[i,:] = (m[:,9]-m[:,8])/m[:,8] * 100

# Now we extract some portion of the last sequences of data for test
X_test=np.zeros([test_length - days_predict , days_history,5])
Y_test=np.zeros([test_length - days_predict , days_predict])
for i in range(test_length - days_predict):
    j = i + train_length - days_history
    m = ticker.history.loc[j : j + days_history - 1].to_numpy()
    X_test[i,:,:] = np.array([(m[:,9]-m[:,8])/m[:,8] * 100 , m[:,7]/1e3 , m[:,6]/1e6 , m[:,2] , m[:,3]]).T
    m = ticker.history.loc[j + days_history : j + days_history - 1 + days_predict].to_numpy()
    Y_test[i,:] = (m[:,9]-m[:,8])/m[:,8] * 100

train_arr = np.arange(train_length - days_history - days_predict - 1)
train_idx = np.random.permutation(train_arr)
X_train = X_train[train_idx]
Y_train = Y_train[train_idx]

test_arr = np.arange(test_length - days_predict - 1)
test_idx = np.random.permutation(test_arr)
X_test = X_test[test_idx]
Y_test = Y_test[test_idx]
X_train = torch.from_numpy(X_train).to(device=Device,dtype=torch.float32)
Y_train = torch.from_numpy(Y_train).to(device=Device,dtype=torch.float32)

X_test = torch.from_numpy(X_test).to(device=Device,dtype=torch.float32)
Y_test = torch.from_numpy(Y_test).to(device=Device,dtype=torch.float32)

In [7]:
class Model(nn.Module):
    def __init__(self,input_size,output_size):
        n=64
        super().__init__()
        self.Lin1 = nn.Linear(input_size,2*n)
        self.Lin2 = nn.Linear(2*n,4*n)
        self.Lin3 = nn.Linear(4*n,2*n)
        self.Lin4 = nn.Linear(2*n,n)
        self.Lin5 = nn.Linear(n,output_size)
        self.activation1 = nn.LeakyReLU(negative_slope=0.1)
        #self.activation2 = nn.Tanh()
        self.drop = nn.Dropout(p=0.5)
        self.norm = nn.BatchNorm1d(4*n)
    def forward(self, x):
        x = self.Lin1(x)
        for i in range(1):
            x_ = x
            x = self.activation1(x)
            x = self.drop(x) ###
            x = self.Lin2(x)
            x = self.norm(x) ####
            x = self.activation1(x)
            x = self.drop(x) ###
            x = self.Lin3(x)
            x = self.activation1(x)
            x = self.drop(x) ###
            x = x + x_

        x = self.Lin4(x)
        x = self.activation1(x)
        x = self.drop(x) ###
        x = self.Lin5(x)
        x = self.activation1(x)
        x = self.drop(x) ###
        return x

model=Model(days_history*5,days_predict).to(device=Device)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
LossFn = nn.MSELoss()
#for p in model.parameters():
#    p.data.fill_(0)

In [8]:
batch_size = 40
train_Losses=[]
test_Losses=[]
N_train = int(np.floor(X_train.size(0)/batch_size))
N_test = int(np.floor(X_test.size(0)/batch_size))

for epoch in range(300):
    model.train()
    train_loss=torch.tensor(0,dtype=torch.float32)
    for i in range(N_train):
        x = torch.flatten(X_train[i*batch_size:(i+1)*batch_size,:,:],start_dim=1)
        y = Y_train[i*batch_size:(i+1)*batch_size,:]
        optimizer.zero_grad()
        out = model(x)
        loss_train = LossFn(y,out)
        loss_train.backward()
        optimizer.step()
        train_loss += loss_train.detach().item()
    train_Losses.append( train_loss/N_train )
   
    model.eval()
    test_loss=torch.tensor(0,dtype=torch.float32)
    test_acc=torch.tensor(0,dtype=torch.float32)
    with torch.no_grad():
        for i in range(N_test):
            x = torch.flatten(X_test[i*batch_size:(i+1)*batch_size,:,:],start_dim=1)
            y = Y_test[i*batch_size:(i+1)*batch_size,:].detach()
            out = model(x)
            loss_test = LossFn(y,out)
            test_loss += loss_test.detach().item()
            temp=y-out
            test_acc += torch.sum(1 - torch.abs(y-out)/2.0).detach().item()
    test_Losses.append( test_loss/N_test )
    accuracy = test_acc / (N_test*batch_size*days_predict) *100
    print(f'{epoch}: train loss is {train_Losses[epoch]} & test loss is {test_Losses[epoch]} & accuracy is %{accuracy}')

0: train loss is 213418.953125 & test loss is 683.7254028320312 & accuracy is %-1155.73681640625
1: train loss is 16470.58203125 & test loss is 252.45220947265625 & accuracy is %-663.0224609375
2: train loss is 5023.56298828125 & test loss is 96.41730499267578 & accuracy is %-362.92034912109375
3: train loss is 2562.736572265625 & test loss is 53.04490280151367 & accuracy is %-237.9883270263672
4: train loss is 1570.24609375 & test loss is 39.484100341796875 & accuracy is %-182.51138305664062
5: train loss is 1410.664306640625 & test loss is 32.99215316772461 & accuracy is %-157.9637451171875
6: train loss is 1125.5460205078125 & test loss is 32.3682746887207 & accuracy is %-155.40513610839844
7: train loss is 1029.1773681640625 & test loss is 30.03194236755371 & accuracy is %-142.7440948486328
8: train loss is 875.9548950195312 & test loss is 23.52391815185547 & accuracy is %-109.6984634399414
9: train loss is 728.3043823242188 & test loss is 23.205217361450195 & accuracy is %-107.302

In [9]:
print(torch.var(Y_train),torch.mean(Y_train))
print(torch.var(Y_test),torch.mean(Y_test))

tensor(9.8854, device='cuda:0') tensor(-0.1974, device='cuda:0')
tensor(4.3821, device='cuda:0') tensor(-0.6184, device='cuda:0')
